In [ ]:
# Just one 
import moxing as mox
mox.file.copy_parallel("obs://jaguars-dataset/dataset1", "test01/")

In [ ]:
!pip install tensorflow==2.9.1

In [ ]:
!pip install -U scikit-learn

In [ ]:
import tensorflow as tf
tf.__version__

In [3]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers

from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix

In [4]:
import imutils
import os
import cv2
import datetime
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer


In [20]:
img_size = (224,224)
batch = 16

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '../test01/train_sample/',
    validation_split = 0.25,
    subset = 'training',
    seed=123,
    image_size= img_size,
    batch_size = batch,
    label_mode = 'binary'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "../test01/val/",
    validation_split = 0.25,
    subset = 'validation',
    seed=123,
    image_size= img_size,
    batch_size = batch,
    label_mode ='binary'
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "../test01/test_sample/",
    #seed=123,
    image_size= img_size,
    batch_size = batch,
    #label_mode = None
)

class_names = test_ds.class_names

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)


preprocess_input = tf.keras.applications.efficientnet.preprocess_input

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomFlip('vertical'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

base_model = tf.keras.applications.EfficientNetB0(input_shape = (224, 224, 3),
                                                    include_top = False,
                                                    weights = 'imagenet')

base_model.trainable = False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1)

inputs = tf.keras.Input(shape=(224,224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = prediction_layer(x)
outputs = tf.nn.sigmoid(x)
model = tf.keras.Model(inputs, outputs)

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy','Recall', 'Precision', 'FalsePositives', 'TruePositives', 'FalseNegatives'])

Found 2800 files belonging to 2 classes.
Using 2100 files for training.
Found 1156 files belonging to 2 classes.
Using 289 files for validation.
Found 800 files belonging to 2 classes.


In [21]:
epochs=100
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  #steps_per_epoch = 100,
  #callbacks = [DisplayCallback()]
)

Epoch 1/100
132/132 [==============================] - 41s 239ms/step - loss: 0.6083 - accuracy: 0.6881 - recall: 0.7363 - precision: 0.6690 - false_positives: 380.0000 - true_positives: 768.0000 - false_negatives: 275.0000 - val_loss: 0.4498 - val_accuracy: 0.8893 - val_recall: 0.8472 - val_precision: 0.7439 - val_false_positives: 21.0000 - val_true_positives: 61.0000 - val_false_negatives: 11.0000
Epoch 2/100
132/132 [==============================] - 28s 215ms/step - loss: 0.4477 - accuracy: 0.8333 - recall: 0.9041 - precision: 0.7904 - false_positives: 250.0000 - true_positives: 943.0000 - false_negatives: 100.0000 - val_loss: 0.3237 - val_accuracy: 0.9170 - val_recall: 0.9583 - val_precision: 0.7667 - val_false_positives: 21.0000 - val_true_positives: 69.0000 - val_false_negatives: 3.0000
Epoch 3/100
132/132 [==============================] - 28s 214ms/step - loss: 0.3768 - accuracy: 0.8633 - recall: 0.9367 - precision: 0.8155 - false_positives: 221.0000 - true_positives: 977.0000

In [22]:
len(val_ds)

19

In [23]:
loss, accuracy, recall, precision, fp, tp, fn = model.evaluate(val_ds)
print('Test accuracy :', accuracy)
print('Test recall :', recall)
print('Test precision :', precision)
print('False Positive :', fp)
print('True Positive :', tp)
print('False Negative :', fn)
tn = len(val_ds)-fp-tp-fn
print('True Negative :', tn)
print('F1 :', (2*(precision*recall))/(precision + recall))

19/19 [==============================] - 3s 168ms/step - loss: 0.1008 - accuracy: 0.9723 - recall: 1.0000 - precision: 0.9000 - false_positives: 8.0000 - true_positives: 72.0000 - false_negatives: 0.0000e+00
Test accuracy : 0.9723183512687683
Test recall : 1.0
Test precision : 0.8999999761581421
False Positive : 8.0
True Positive : 72.0
False Negative : 0.0
True Negative : -61.0
F1 : 0.9473684078438459


In [30]:
model.save('../train_w_model02A/wHC_1_1.h5')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
acc = history.history['recall']
val_acc = history.history['val_recall']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Recall')
plt.plot(epochs_range, val_acc, label='Validation Recall')
plt.legend(loc='lower right')
plt.title('Training and Validation Recall')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
acc = history.history['precision']
val_acc = history.history['val_precision']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training precision')
plt.plot(epochs_range, val_acc, label='Validation precision')
plt.legend(loc='lower right')
plt.title('Training and Validation precision')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [27]:
len(test_ds)

50

In [28]:
loss, accuracy, recall, precision, fp, tp, fn = model.evaluate(test_ds)
print('Test accuracy :', accuracy)
print('Test recall :', recall)
print('Test precision :', precision)
print('False Positive :', fp)
print('True Positive :', tp)
print('False Negative :', fn)
tn = len(test_ds)-fp-tp-fn
print('True Negative :', tn)
print('F1 :', (2*(precision*recall))/(precision + recall))

50/50 [==============================] - 12s 183ms/step - loss: 0.1073 - accuracy: 0.9675 - recall: 0.9750 - precision: 0.9606 - false_positives: 16.0000 - true_positives: 390.0000 - false_negatives: 10.0000
Test accuracy : 0.9674999713897705
Test recall : 0.9750000238418579
Test precision : 0.9605911374092102
False Positive : 16.0
True Positive : 390.0
False Negative : 10.0
True Negative : -366.0
F1 : 0.9677419494630279


In [29]:
loss, accuracy, recall, precision, fp, tp, fn = model.evaluate(train_ds)
print('Test accuracy :', accuracy)
print('Test recall :', recall)
print('Test precision :', precision)
print('False Positive :', fp)
print('True Positive :', tp)
print('False Negative :', fn)
tn = len(train_ds)-fp-tp-fn
print('True Negative :', tn)
print('F1 :', (2*(precision*recall))/(precision + recall))

132/132 [==============================] - 23s 172ms/step - loss: 0.1056 - accuracy: 0.9657 - recall: 0.9732 - precision: 0.9585 - false_positives: 44.0000 - true_positives: 1015.0000 - false_negatives: 28.0000
Test accuracy : 0.9657142758369446
Test recall : 0.9731543660163879
Test precision : 0.9584513902664185
False Positive : 44.0
True Positive : 1015.0
False Negative : 28.0
True Negative : -955.0
F1 : 0.9657469201657135


In [ ]:
# 128 x 128 Model - val_generator
'''
10/10 [==============================] - 1s 115ms/step - loss: 0.1810 - accuracy: 0.9170 - recall: 0.8889 - precision: 0.8000 - false_positives: 16.0000 - true_positives: 64.0000 - false_negatives: 8.0000
Test accuracy : 0.9169549942016602
Test recall : 0.8888888955116272
Test precision : 0.800000011920929
False Positive : 16.0
True Positive : 64.0
False Negative : 8.0
True Negative : -78.0
F1 : 0.8421052727342643
'''

In [ ]:
# 128 x 128 Model - train_generator
'''
66/66 [==============================] - 8s 124ms/step - loss: 0.1856 - accuracy: 0.9352 - recall: 0.9473 - precision: 0.9242 - false_positives: 81.0000 - true_positives: 988.0000 - false_negatives: 55.0000
Test accuracy : 0.9352381229400635
Test recall : 0.947267472743988
Test precision : 0.9242282509803772
False Positive : 81.0
True Positive : 988.0
False Negative : 55.0
True Negative : -1058.0
F1 : 0.9356060486235137
'''

In [ ]:
# 128 x 128 Model - test_generator
'''
25/25 [==============================] - 4s 139ms/step - loss: 0.1913 - accuracy: 0.9312 - recall: 0.9250 - precision: 0.9367 - false_positives: 25.0000 - true_positives: 370.0000 - false_negatives: 30.0000
Test accuracy : 0.9312499761581421
Test recall : 0.925000011920929
Test precision : 0.9367088675498962
False Positive : 25.0
True Positive : 370.0
False Negative : 30.0
True Negative : -400.0
F1 : 0.9308176194511963
'''

In [ ]:
# 224 x 224 Model - val_generator
'''
19/19 [==============================] - 3s 168ms/step - loss: 0.1008 - accuracy: 0.9723 - recall: 1.0000 - precision: 0.9000 - false_positives: 8.0000 - true_positives: 72.0000 - false_negatives: 0.0000e+00
Test accuracy : 0.9723183512687683
Test recall : 1.0
Test precision : 0.8999999761581421
False Positive : 8.0
True Positive : 72.0
False Negative : 0.0
True Negative : -61.0
F1 : 0.9473684078438459
'''

In [ ]:
# 224 x 224 Model - train_generator
'''
132/132 [==============================] - 23s 172ms/step - loss: 0.1056 - accuracy: 0.9657 - recall: 0.9732 - precision: 0.9585 - false_positives: 44.0000 - true_positives: 1015.0000 - false_negatives: 28.0000
Test accuracy : 0.9657142758369446
Test recall : 0.9731543660163879
Test precision : 0.9584513902664185
False Positive : 44.0
True Positive : 1015.0
False Negative : 28.0
True Negative : -955.0
F1 : 0.9657469201657135
'''

In [ ]:
# 224 x 224 Model - test_generator
'''
50/50 [==============================] - 12s 183ms/step - loss: 0.1073 - accuracy: 0.9675 - recall: 0.9750 - precision: 0.9606 - false_positives: 16.0000 - true_positives: 390.0000 - false_negatives: 10.0000
Test accuracy : 0.9674999713897705
Test recall : 0.9750000238418579
Test precision : 0.9605911374092102
False Positive : 16.0
True Positive : 390.0
False Negative : 10.0
True Negative : -366.0
F1 : 0.9677419494630279
'''